Based on https://ymd_h.gitlab.io/ymd_blog/posts/gym_on_google_colab_with_gnwrapper/

## Setup

In [ ]:
!sudo apt-get install -y xvfb ffmpeg x11-utils
!pip install -q 'gym==0.10.11'
!pip install -q 'imageio==2.4.0'
!pip install -q PILLOW
!pip install -q 'pyglet==1.3.2'
!pip install -q pyvirtualdisplay
!pip install -q tf-agents
!pip install colabgymrender
!pip install gym_sokoban
!pip install gym
!apt-get install python-opengl -y
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install piglet
import gym
import gym_sokoban

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

from colabgymrender.recorder import Recorder
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
Display().start()
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

In [ ]:
env = gym.make("PushAndPull-Sokoban-v2")
screen = env.render(mode='rgb_array')
plt.imshow(screen)
observation = env.reset()
ipythondisplay.clear_output(wait=True)
ipythondisplay.display(plt.gcf())

In [ ]:
observation.shape

In [ ]:
from skimage import io, color

lina_gray = color.rgb2gray(observation[16:96, 16:96])
print(lina_gray.shape)
plt.imshow(lina_gray)

In [ ]:
d = Display()
d.start()

screeens = list()
directory = './video'
env = gym.wrappers.Monitor(env, "recording",force=True)
env.render(mode='human')
env.reset()
for _ in range(2):
    action = env.action_space.sample()
    #print(action)
    print(action)
    state, reward, done, info = env.step(action) 
    print(state.shape, reward, done, info)
    screen = env.render(mode='rgb_array')
    screeens.append(screen)
    #plt.imshow(screen)
    #ipythondisplay.clear_output(wait=True)
    #ipythondisplay.display(plt.gcf())
    if done:
        env.reset()
        print('done')

In [ ]:
plt.imshow(screen)
ipythondisplay.clear_output(wait=True)
ipythondisplay.display(plt.gcf())

In [ ]:
import gym
import random
import os
import numpy as np
from collections      import deque
from keras.models     import Sequential
from keras.layers     import Dense, Conv2D, Flatten
from keras.optimizers import Adam

In [ ]:
class Agent():
    def __init__(self, state_size, action_size):
        self.weight_backup      = "SOKOBAN_weight.h5"
        self.state_size         = state_size
        self.action_size        = action_size
        self.memory             = deque(maxlen=2000)
        self.learning_rate      = 0.001 # @param {type:"number"}
        self.gamma              = 0.95 # @param {type:"number"}
        self.exploration_rate   = 1.0# @param {type:"number"}
        self.exploration_min    = 0.01# @param {type:"number"}
        self.exploration_decay  = 0.995 # @param {type:"number"}
        self.brain              = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=self.state_size, padding='same'))
        model.add(Dense(32, activation='relu'))
        #model.add(Dense(16, activation='relu'))
        #model.add(Flatten())
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        '''if os.path.isfile(self.weight_backup):
            model.load_weights(self.weight_backup)
            self.exploration_rate = self.exploration_min'''
        return model

    def save_model(self):
            self.brain.save(self.weight_backup)

    def act(self, state):
        if np.random.rand() <= self.exploration_rate:
            return random.randrange(self.action_size)
        act_values = self.brain.predict(state)
        return np.argmax(act_values[0][0][0])

    def remember(self, state, action, reward, next_state, done, info):
        self.memory.append((state, action, reward, next_state, done, info))

    def replay(self, sample_batch_size):
        if len(self.memory) < sample_batch_size:
            return
        sample_batch = random.sample(self.memory, sample_batch_size)
        for state, action, reward, next_state, done, info in sample_batch:
            target = reward

            if not done:
              target = reward + self.gamma * np.amax(self.brain.predict(next_state)[0])
            target_f = self.brain.predict(state)
            target_f[0][action] = target
            self.brain.fit(next_state, target_f, epochs=1, verbose=0)
        if self.exploration_rate > self.exploration_min:
            self.exploration_rate *= self.exploration_decay

In [ ]:
class Sokoban:
    def __init__(self):
        self.sample_batch_size = 64 # @param {type:"integer"}
        self.episodes          =  300000# @param {type:"integer"}
        self.env               = gym.make('PushAndPull-Sokoban-v2')

        self.state_size        =  (80, 80, 1)
        self.action_size       = self.env.action_space.n
        self.agent             = Agent(self.state_size, self.action_size)


    def run(self):
        try:
            for index_episode in range(self.episodes):
                state = self.env.reset()
                state = color.rgb2gray(state[16:96, 16:96]).reshape(1, 80, 80, 1)

                done = False
                index = 0
                total_reward = list()
                while not done:
#                    self.env.render()
                    action = self.agent.act(state)
                    next_state, reward, done, info = self.env.step(action)
                    reward = reward*10

                    total_reward.append(reward)
                    next_state = color.rgb2gray(next_state[16:96, 16:96]).reshape(1, 80, 80, 1)
                    self.agent.remember(state, action, reward, next_state, done, info)
                    state = next_state
                    index += 1
                print("Episode {}# steps: {}, total reward: {}, last reward: {}, rewards_list: {}".format(index_episode, index + 1, sum(total_reward), reward, total_reward))
                self.agent.replay(self.sample_batch_size)
        finally:
            self.agent.save_model()


In [ ]:
if __name__ == "__main__":
    sokoban = Sokoban()
    sokoban.run()